<a href="https://colab.research.google.com/github/Tamanna1991/neural-networks/blob/main/lab4/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
# There are 24,983 rows each represent a user
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv", header=None)
original_df = df.copy()
df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
24981,37,99.00,99.00,99.00,99.00,-9.71,99.00,4.56,-8.30,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00


In [3]:
# Ratings are real values ranging from -10.00 to +10.00. Replace the value "99" with NaN because it corresponds to "not rated".
df = df.replace(99.00, np.NaN)
# df = df.applymap(lambda x: np.nan if x == 99.00 else x)
df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,74.0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,100.0,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49.0,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,48.0,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,91.0,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100.0,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91.0,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39.0,NaN,NaN,NaN,NaN,-7.77,NaN,6.70,-6.75,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24981,37.0,NaN,NaN,NaN,NaN,-9.71,NaN,4.56,-8.30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Drop the first column because it represents the number of jokes rated by that user while keeping the next 100 columns because they're the ratings for jokes 01 - 100.
df.drop(0, inplace=True, axis=1)
df

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,NaN,NaN,NaN,NaN,-7.77,NaN,6.70,-6.75,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24981,NaN,NaN,NaN,NaN,-9.71,NaN,4.56,-8.30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.describe() 

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
count,16452.000000,18184.000000,16642.000000,15608.000000,24977.000000,20431.000000,24976.000000,24981.000000,15752.000000,20324.000000,...,9672.000000,9889.000000,9988.000000,10200.000000,10265.000000,10370.000000,10520.000000,10666.000000,10910.000000,9943.000000
mean,0.904573,0.208460,0.316574,-1.449114,0.403556,1.594447,-0.429662,-0.624085,-0.557031,1.301231,...,2.038527,1.179744,2.504041,1.173482,1.039130,1.528003,1.673750,0.767692,-0.031244,1.354711
std,5.171798,5.597327,5.399806,5.234654,5.255700,4.900540,5.495931,4.929144,5.162224,5.087359,...,5.060445,5.127291,4.611339,5.140307,5.167043,4.852069,5.025076,5.393833,5.176806,5.300409
min,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,...,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000
25%,-2.670000,-4.420000,-4.030000,-5.870000,-3.690000,-1.310000,-5.290000,-4.610000,-4.760000,-2.090000,...,-0.932500,-2.380000,-0.100000,-2.280000,-2.570000,-1.347500,-1.170000,-3.400000,-4.170000,-2.330000
50%,1.600000,0.830000,0.780000,-1.460000,1.120000,2.230000,-0.050000,-0.290000,-0.190000,1.940000,...,2.860000,1.800000,3.160000,1.750000,1.650000,2.140000,2.480000,1.460000,0.290000,2.140000
75%,4.900000,4.820000,4.660000,2.520000,4.470000,5.440000,3.880000,3.060000,3.300000,5.390000,...,6.120000,5.290000,6.070000,5.290000,5.100000,5.277500,5.630000,5.150000,3.980000,5.730000
max,9.370000,9.370000,9.370000,9.370000,9.370000,9.470000,9.610000,9.760000,9.900000,9.370000,...,9.560000,9.420000,9.370000,9.420000,9.370000,9.370000,9.370000,9.370000,9.370000,9.370000


In [6]:
#  Use pandas to find the best- and the worst-rated jokes
joke_avg_rating = df.mean(axis=0)
joke_avg_rating

1      0.904573
2      0.208460
3      0.316574
4     -1.449114
5      0.403556
         ...   
96     1.528003
97     1.673750
98     0.767692
99    -0.031244
100    1.354711
Length: 100, dtype: float64

In [7]:
joke_avg_rating[1]

0.9045733041575473

In [8]:
joke_avg_rating[[1]]

1    0.904573
dtype: float64

In [9]:
joke_avg_rating.shape

(100,)

In [10]:
joke_avg_rating.max()

3.6650848950824937

In [11]:
joke_avg_rating.min()

-3.8338796373689923

In [12]:
joke_avg_rating[joke_avg_rating==3.6650848950824937].index.item()

50

In [13]:
max_index = joke_avg_rating.idxmax()
print("The best rated joke id:", max_index)

The best rated joke id: 50


In [14]:
min_index = joke_avg_rating.idxmin()
min_index
print("The worst rated joke id:", min_index)

The worst rated joke id: 58


In [15]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  # rated = np.where(arr!=99)
  rated = np.where(~np.isnan(orig))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [16]:
arr = df.values
print(arr.shape)
arr

(24983, 100)


array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [ 2.43,  2.67, -3.98, ...,   nan,   nan,   nan]])

In [17]:
new_arr, idx = replace(arr, 0.1)
print(new_arr.shape)
new_arr

(24983, 100)


array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [ 2.43,  2.67, -3.98, ...,   nan,   nan,   nan]])

In [18]:
idx

(array([ 7234, 22912,  5364, ...,  3729, 23156,   756]),
 array([44, 51,  7, ..., 72, 11, 47]))

In [19]:
arr[idx[0][0], idx[1][0]]

8.06

In [20]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [21]:
n_latent_factors = 2

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [22]:
print(latent_user_preferences.shape)
latent_user_preferences

(24983, 2)


array([[0.54784024, 0.56982762],
       [0.14470721, 0.30199999],
       [0.47046276, 0.49790544],
       ...,
       [0.14868621, 0.72517053],
       [0.1117665 , 0.3659026 ],
       [0.45207962, 0.739644  ]])

In [23]:
print(latent_item_features.shape)
latent_item_features

(100, 2)


array([[0.61223831, 0.39587123],
       [0.65276707, 0.49059774],
       [0.96021588, 0.46442191],
       [0.29415518, 0.74018768],
       [0.74663528, 0.93623524],
       [0.80960565, 0.88182744],
       [0.60013398, 0.69581191],
       [0.86794893, 0.45938217],
       [0.58836936, 0.4002793 ],
       [0.34516843, 0.77803775],
       [0.55578227, 0.0558851 ],
       [0.81497783, 0.35164071],
       [0.88917761, 0.67271253],
       [0.6515167 , 0.3816569 ],
       [0.11452331, 0.17962484],
       [0.94511251, 0.30579151],
       [0.44518306, 0.67307097],
       [0.18591479, 0.19421462],
       [0.81865613, 0.87208746],
       [0.89353532, 0.59692643],
       [0.53941506, 0.02292678],
       [0.55857573, 0.12498669],
       [0.6023072 , 0.40899173],
       [0.70260202, 0.18378998],
       [0.5572526 , 0.26104441],
       [0.3469577 , 0.98430457],
       [0.67158038, 0.06615591],
       [0.92920487, 0.15777826],
       [0.54108226, 0.16493526],
       [0.33387008, 0.59933036],
       [0.

In [24]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = new_arr[user_id, item_id]
                if rating != 99 and not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [ ]:
num_iter = 100
hist = sgd(num_iter)  # Note how the MSE decreases with the number of iterations
plt.figure()
plt.plot(np.arange(0, num_iter, 10000), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

In [26]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 0.22273504, -0.04237737, -0.09964628, ...,  0.62620248,
         0.09397781,  0.98753675],
       [ 1.09517103,  1.03247412,  0.98996101, ...,  1.21235653,
         0.81840668,  1.41801259],
       [ 3.60881684,  3.24335948,  3.07266266, ...,  4.23394217,
         2.65120228,  5.1127478 ],
       ...,
       [-0.14142757, -0.36634173, -0.40574605, ...,  0.19396425,
        -0.17259928,  0.46241529],
       [ 0.51546798,  0.33102837,  0.28116812, ...,  0.8036896 ,
         0.34071202,  1.09663921],
       [ 2.31197051,  1.95573321,  1.82285033, ...,  2.8961459 ,
         1.66341421,  3.61374729]])

In [27]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns
comparison_data

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, 0.2227350435598823)","(8.79, -0.04237736815617642)","(-9.66, -0.09964627747529833)","(-8.16, -1.2300421886752693)","(-7.52, 0.0504495961119938)","(-8.5, 0.959504622996773)","(-9.85, -0.5747492653409275)","(4.17, -0.6662978591790827)","(-8.98, -0.882930724660853)","(-4.76, 0.437805282748956)",...,"(2.82, 1.407242347673782)","(nan, 0.8686438756826709)","(nan, 1.8260884718681285)","(nan, 0.5770173639648387)","(nan, 0.724591447986671)","(nan, 1.0481900821259071)","(-5.63, 1.1893298913932946)","(nan, 0.6262024839403372)","(nan, 0.09397780769762382)","(nan, 0.987536753827125)"
1,"(4.08, 1.0951710331937026)","(-0.29, 1.0324741161178228)","(6.36, 0.989961013994145)","(4.37, 0.3801014821045897)","(-2.38, 0.8604382888041198)","(-9.66, 1.6436533907762063)","(-0.73, 0.48299261500170426)","(-5.34, 0.13158775868933623)","(8.88, 0.694484917216864)","(9.22, 1.2511372852759908)",...,"(2.82, 1.4546416149155061)","(-4.95, 1.2854698022570752)","(-0.29, 1.656532122414267)","(7.86, 1.1908159139784504)","(-0.19, 1.104181137233681)","(-2.14, 1.3316290012866148)","(3.06, 1.3106546460051944)","(0.34, 1.2123565256232103)","(-4.32, 0.8184066810155111)","(1.07, 1.4180125860732944)"
2,"(nan, 3.608816843415726)","(nan, 3.2433594817979903)","(nan, 3.0726626596632687)","(nan, 0.42955633382764524)","(9.03, 2.7569236915750115)","(9.27, 5.8097478946002905)","(9.03, 1.1679295463096842)","(9.27, -0.0026628859318000363)","(nan, 1.6437684980198661)","(nan, 4.238174473849395)",...,"(nan, 5.492957694232246)","(nan, 4.618118960534829)","(nan, 6.396040064076607)","(9.08, 4.134757723457497)","(nan, 3.9532653453907147)","(nan, 4.877335553387373)","(nan, 4.899751519999189)","(nan, 4.233942173579824)","(nan, 2.6512022801205966)","(nan, 5.11274780493397)"
3,"(nan, 2.0656784599949085)","(8.35, 1.8890958198715346)","(nan, 1.7976712506660466)","(nan, 0.41478240158319796)","(1.8, 1.5941477119571237)","(8.16, 3.244709806620772)","(-2.82, 0.7554671957655105)","(6.21, 0.08801707522333926)","(nan, 1.0732096173838537)","(1.84, 2.402233051536659)",...,"(nan, 3.0005669672691866)","(nan, 2.564951455173051)","(nan, 3.4686804358630963)","(0.53, 2.3234667498189503)","(nan, 2.1982383816059694)","(nan, 2.691342478445429)","(nan, 2.685387359675201)","(nan, 2.3744511303561384)","(nan, 1.5269048812337203)","(nan, 2.836197954216358)"
4,"(8.5, 3.344651643733675)","(4.61, 3.1260721679977435)","(-4.17, 2.9910151990544516)","(-5.39, 1.0204229985074145)","(1.36, 2.6144020104238876)","(1.6, 5.086863873837115)","(7.04, 1.402782318069124)","(4.61, 0.32743553779360907)","(-0.44, 2.0109620127925543)","(5.73, 3.8406639301090224)",...,"(5.19, 4.561836749720803)","(5.58, 3.991036788018398)","(4.27, 5.218983739962837)","(5.19, 3.672711198271928)","(5.73, 3.4258701588850298)","(1.55, 4.150281654266145)","(3.11, 4.101842279415667)","(6.55, 3.7433081031244972)","(1.8, 2.491630435620737)","(1.6, 4.405696171629603)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,"(0.44, 2.8132906911317206)","(7.43, 2.481589091372962)","(9.08, 2.339502553051713)","(2.33, 0.09239100703827541)","(3.2, 2.126088995718226)","(6.75, 4.6450090866195035)","(-8.79, 0.7856537944626102)","(-0.53, -0.13061797816260565)","(-8.74, 1.091462809525367)","(7.23, 3.3379189541357754)",...,"(8.83, 4.488223745391027)","(-1.21, 3.712721364744205)","(9.22, 5.262299059965021)","(-6.7, 3.285394436610185)","(8.45, 3.1745487185813746)","(9.03, 3.94635569049735)","(6.55, 3.9907987105623763)","(8.69, 3.3710245580434623)","(8.79, 2.0533314296810765)","(7.43, 4.115365905004767)"
24979,"(9.13, 1.6778425012142402)","(-8.16, 1.708487619122229)","(8.59, 1.6677671949123485)","(9.08, 1.2386892206022069)","(0.87, 1.3807520086183023)","(-8.93, 2.2042432640355045)","(-3.5, 1.0778352901072834)","(5.78, 0.5495513931686388)","(-8.11, 1.5781682215633686)","(4.9, 1.8247368597573481)",...,"(-1.17, 1.6705818784198676)","(-5.73, 1.6645366027915889)","(-1.46, 1.790216

# Quiz

In [29]:
# In the jokes dataset, what's the rating that user 9213 has given to joke 7?
df.iloc[9213, 7]

0.83

In [33]:
mov_items = pd.read_excel('movies_latent_factors.xlsx', sheet_name='Items')
mov_items

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [34]:
mov_users = pd.read_excel('movies_latent_factors.xlsx', sheet_name='Users')
mov_users

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
1,156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
3,174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4,4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669
5,783,-0.020301,-0.031919,-0.036955,0.033690,0.000174,-0.003178,0.054474,0.045424,-0.053308,0.003437,-0.041750,0.059725,-0.021647,0.039873,-0.061857
6,3878,-0.091462,0.215879,-0.180453,0.085408,-0.321094,0.227947,0.053767,0.038017,0.097141,-0.139872,0.029253,0.214467,0.189250,0.065990,0.102776
7,768,0.000819,-0.009229,-0.019228,0.002703,0.012869,0.006655,0.007687,0.018747,0.014705,-0.009256,-0.010116,-0.010051,-0.002108,0.043223,-0.006670
8,4469,-0.030528,-0.011537,-0.042822,-0.014378,0.031338,0.012297,-0.000038,-0.010264,-0.006781,-0.019950,-0.014435,0.010926,0.024400,0.005318,-0.007983
9,1882,-0.083093,-0.029160,0.013748,0.022716,-0.062732,-0.080670,-0.019723,0.013253,-0.091305,-0.090687,0.030638,-0.113364,-0.021325,0.181508,0.022357


In [35]:
# In the movies dataset, what's the loading (i.e., value) of Factor13 for user with ID=4768?
mov_users.loc[mov_users['User'] == 2024, 'Factor2']

Series([], Name: Factor2, dtype: float64)

In [36]:
# In the movies dataset, what's the loading (i.e., value) of Factor4 for the movie with ID=857?
mov_items.loc[mov_items['Movie ID'] == 2024, 'Factor2']

81   -0.497786
Name: Factor2, dtype: float64

In [ ]:
# In the movies dataset, which of the following titles corresponds to the movie with the lowest loading of factor 1?
mov_items.loc[mov_items['Factor8'] == mov_items['Factor8'].min() , 'Movie ID']  #  'Title'

30    272
Name: Movie ID, dtype: int64

In [38]:
# In the movies dataset, which of the following titles corresponds to the movie with the highest loading of factor 10?
mov_items.loc[mov_items['Factor6'] == mov_items['Factor6'].max() , 'Movie ID']

90    8467
Name: Movie ID, dtype: int64

In the movies dataset, predict the rating that the user with ID 525 will give to the movie with ID 585. Give your answer with two decimal values (e.g., 0.12).

In [82]:
mov_factor_vals = mov_items.loc[mov_items['Movie ID'] == 9741]
mov_factor_vals

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
93,9741,Unbreakable (2000),0.873408,0.560229,-0.120818,-0.203702,0.425619,-0.261493,-0.342108,-0.573046,-0.125282,0.609137,0.16689,-0.08996,-0.040781,-0.327417,0.730784


In [83]:
mov_factor_vals.drop('Title', inplace=True, axis=1)
mov_factor_vals.drop('Movie ID', inplace=True, axis=1)
mov_factor_vals

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
93,0.873408,0.560229,-0.120818,-0.203702,0.425619,-0.261493,-0.342108,-0.573046,-0.125282,0.609137,0.16689,-0.08996,-0.040781,-0.327417,0.730784


In [84]:
user_factor_vals = mov_users.loc[mov_users['User'] == 156]
user_factor_vals

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
1,156,-0.189652,-0.178979,-0.09149,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873


In [85]:
user_factor_vals.drop('User', inplace=True, axis=1)
user_factor_vals

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
1,-0.189652,-0.178979,-0.09149,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873


In [86]:
predictions = user_factor_vals.dot(mov_factor_vals.T)
predictions

,93
1,-0.072079


In the movies dataset, predict the rating that the user with ID 3806 will give to the movie with ID 24. Give your answer with two decimal values (e.g., 0.12).

In [ ]:
mov_factor_vals = mov_items.loc[mov_items['Movie ID'] == 1597]
mov_factor_vals

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
75,1597,Meet the Parents (2000),1.569911,0.540649,-0.685263,0.091106,0.550054,0.574014,0.44043,-0.341092,-0.158271,0.677886,-0.351136,1.192418,-0.276798,-0.017176,0.194981


In [ ]:
mov_factor_vals.drop('Title', inplace=True, axis=1)
mov_factor_vals.drop('Movie ID', inplace=True, axis=1)
mov_factor_vals

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
75,1.569911,0.540649,-0.685263,0.091106,0.550054,0.574014,0.44043,-0.341092,-0.158271,0.677886,-0.351136,1.192418,-0.276798,-0.017176,0.194981


In [ ]:
user_factor_vals = mov_users.loc[mov_users['User'] == 3048]
user_factor_vals

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
22,3048,-0.320315,0.07829,-0.083182,-0.221577,-0.078184,-0.145306,-0.020166,0.124409,-0.063425,0.004726,0.009683,0.028016,-0.0181,-0.128452,0.069972


In [ ]:
user_factor_vals.drop('User', inplace=True, axis=1)
user_factor_vals

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
22,-0.320315,0.07829,-0.083182,-0.221577,-0.078184,-0.145306,-0.020166,0.124409,-0.063425,0.004726,0.009683,0.028016,-0.0181,-0.128452,0.069972


In [ ]:
predictions = user_factor_vals.dot(mov_factor_vals.T)
predictions

,75
22,-0.537347


# In the movies dataset, if you had to recommend 3 movies to the user with ID=5323, which ones would you choose?

In [46]:
user_factor_vals = mov_users.loc[mov_users['User'] == 3947]
user_factor_vals

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
17,3947,-0.02234,-0.115285,-0.13698,0.038593,-0.003029,0.07177,-0.146675,-0.285724,-0.044472,-0.054032,0.064019,-0.045329,-0.111919,0.066313,0.002395


In [43]:
user_factor_vals.drop('User', inplace=True, axis=1)
user_factor_vals

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
20,-0.04029,0.014383,-0.098967,-0.018957,0.012388,0.035546,0.000903,0.012835,-0.02108,0.018742,-0.050483,-0.025394,0.034079,0.03833,-0.001341


In [47]:
mov_items_arr = mov_items.copy()
mov_items_arr

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [45]:
mov_items_arr.drop('Title', inplace=True, axis=1)
mov_items_arr.drop('Movie ID', inplace=True, axis=1)
mov_items_arr

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [50]:
predictions = mov_items_arr.dot(user_factor_vals.T)
predictions

ValueError: ignored

In [49]:
largest = predictions.stack().nlargest(3)
largest

AttributeError: ignored

In [51]:
mov_items

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [52]:
mov_items.loc[51]

Movie ID                           604
Title       The Matrix Reloaded (2003)
Factor1                       0.503414
Factor2                       -2.00445
Factor3                       0.001084
Factor4                      -2.255945
Factor5                       0.882547
Factor6                        -0.1886
Factor7                      -0.235434
Factor8                       0.570028
Factor9                      -3.492897
Factor10                     -0.312214
Factor11                      0.902742
Factor12                     -2.063318
Factor13                     -1.895101
Factor14                      0.748458
Factor15                      2.120907
Name: 51, dtype: object

In [ ]:
mov_items.loc[50]

Movie ID                  603
Title       The Matrix (1999)
Factor1              -3.06364
Factor2              -1.31928
Factor3            -0.0436179
Factor4             -0.994612
Factor5             -0.954054
Factor6              0.190838
Factor7              0.197918
Factor8            -0.0939622
Factor9              -1.08284
Factor10          -0.00384283
Factor11              1.37745
Factor12             -1.66819
Factor13             -1.34838
Factor14             0.223797
Factor15             0.752413
Name: 50, dtype: object

In [ ]:
mov_items.loc[22]

Movie ID                      161
Title       Ocean's Eleven (2001)
Factor1                 -0.169048
Factor2                 -0.253796
Factor3                 -0.703979
Factor4                 -0.988572
Factor5                 -0.527166
Factor6                 -0.818696
Factor7                  0.283189
Factor8                  -0.21164
Factor9                 -0.518536
Factor10                 0.911899
Factor11                0.0908231
Factor12                 0.484479
Factor13                 0.328734
Factor14                  1.62584
Factor15                -0.575322
Name: 22, dtype: object

# In the movies dataset, if you had to recommend 2 movies to the user with ID=525, which ones would you choose?

In [63]:
user_factor_vals = mov_users.loc[mov_users['User'] == 3947]
user_factor_vals

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
17,3947,-0.02234,-0.115285,-0.13698,0.038593,-0.003029,0.07177,-0.146675,-0.285724,-0.044472,-0.054032,0.064019,-0.045329,-0.111919,0.066313,0.002395


In [64]:
user_factor_vals.drop('User', inplace=True, axis=1)
user_factor_vals

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
17,-0.02234,-0.115285,-0.13698,0.038593,-0.003029,0.07177,-0.146675,-0.285724,-0.044472,-0.054032,0.064019,-0.045329,-0.111919,0.066313,0.002395


In [65]:
mov_items_arr = mov_items.copy()
mov_items_arr

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [66]:
mov_items_arr.drop('Title', inplace=True, axis=1)
mov_items_arr.drop('Movie ID', inplace=True, axis=1)
mov_items_arr

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [67]:
predictions = mov_items_arr.dot(user_factor_vals.T)
predictions

,17
0,0.181496
1,0.054192
2,0.145365
3,0.056495
4,-0.055989
...,...
95,0.021683
96,-0.164788
97,0.027508
98,0.049276


In [68]:
largest = predictions.stack().nlargest(3)
largest

50  17    0.582245
51  17    0.578413
39  17    0.431327
dtype: float64

In [69]:
mov_items.loc[50]

Movie ID                  603
Title       The Matrix (1999)
Factor1             -3.063639
Factor2             -1.319283
Factor3             -0.043618
Factor4             -0.994612
Factor5             -0.954054
Factor6              0.190838
Factor7              0.197918
Factor8             -0.093962
Factor9              -1.08284
Factor10            -0.003843
Factor11             1.377453
Factor12            -1.668189
Factor13            -1.348377
Factor14             0.223797
Factor15             0.752413
Name: 50, dtype: object

In [70]:
mov_items.loc[51]

Movie ID                           604
Title       The Matrix Reloaded (2003)
Factor1                       0.503414
Factor2                       -2.00445
Factor3                       0.001084
Factor4                      -2.255945
Factor5                       0.882547
Factor6                        -0.1886
Factor7                      -0.235434
Factor8                       0.570028
Factor9                      -3.492897
Factor10                     -0.312214
Factor11                      0.902742
Factor12                     -2.063318
Factor13                     -1.895101
Factor14                      0.748458
Factor15                      2.120907
Name: 51, dtype: object

In [62]:
mov_items.loc[39]

Movie ID                        453
Title       A Beautiful Mind (2001)
Factor1                   -1.718302
Factor2                   -0.626027
Factor3                   -2.113008
Factor4                   -0.234649
Factor5                     1.07486
Factor6                   -0.368105
Factor7                    -0.31073
Factor8                    0.156776
Factor9                   -0.951447
Factor10                  -0.213479
Factor11                   0.366553
Factor12                   0.050671
Factor13                  -0.080249
Factor14                  -0.224858
Factor15                   0.078898
Name: 39, dtype: object